In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-2.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-2        9    6    5    0   13   16     9   1  12  ...  0.000000   
1      prop-2        9    4    4    0   14   20     4   1  13  ...  0.400000   
2      prop-2        9    6    4    0   28   41    15   1  27  ...  0.000000   
3      prop-2        9    2    3    0   16   13     1   1  15  ...  0.000000   
4      prop-2        9    2    3    0   17   19     1   1  16  ...  0.000000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
23009  prop-2      192    2    3    0   16   24     1   0  16  ...  0.000000   
23010  prop-2      192    4    5    0    9   13     4   0   9  ...  0.000000   
23011  prop-2      192   11    1    0   41  113    27   1  40  ...  0.909091   
23012  prop-2      192    2    3    0    9   11     1   0   9  ...  1.000000   
23013  prop-2      192    6    6    0   17   22     3   0  17  ...  0.000000   

       moa       mfa       cam  ic  cbm         amc  max_cc   avg_cc  bug  
0        0  0.846154  0.440000   2    4    8.000000       1   0.6667    0  
1        0  0.959184  0.555556   3    5   18.250000       1   0.5000    0  
2        0  0.857143  0.404762   1    3   26.333333       5   1.5000    0  
3        0  0.875000  0.625000   1    1   40.000000       7   3.5000    0  
4        0  0.875000  0.625000   1    1   47.000000       1   0.5000    0  
...    ...       ...       ...  ..  ...         ...     ...      ...  ...  
23009    0  0.875000  0.625000   1    1  109.000000      27  13.5000    0  
23010    0  0.913043  0.555556   2    2    9.000000       1   0.5000    0  
23011    1  0.000000  0.363636   0    0  212.272727      15   3.5455    2  
23012    0  0.875000  0.625000   1    1   30.000000       5   2.5000    0  
23013    0  0.840000  0.733333   2    3   12.666667       1   0.6667    0  

[23014 rows x 23 columns]

In [5]:
ant_master = ant_master.loc[ant_master['version'] == 9]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-2        9    6    5    0   13   16     9   1  12  ...  0.000000   
1     prop-2        9    4    4    0   14   20     4   1  13  ...  0.400000   
2     prop-2        9    6    4    0   28   41    15   1  27  ...  0.000000   
3     prop-2        9    2    3    0   16   13     1   1  15  ...  0.000000   
4     prop-2        9    2    3    0   17   19     1   1  16  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
4250  prop-2        9    4    7    0    9   13     4   0   9  ...  0.000000   
4251  prop-2        9    2    3    0   15   15     1   1  14  ...  0.000000   
4252  prop-2        9    5    6    0    3    9    10   1   2  ...  0.000000   
4253  prop-2        9    4    4    0   10   21     4   0  10  ...  0.333333   
4254  prop-2        9    2    2    0   11   14     1   1  10  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.846154  0.440000   2    4   8.000000       1  0.6667    0  
1       0  0.959184  0.555556   3    5  18.250000       1  0.5000    0  
2       0  0.857143  0.404762   1    3  26.333333       5  1.5000    0  
3       0  0.875000  0.625000   1    1  40.000000       7  3.5000    0  
4       0  0.875000  0.625000   1    1  47.000000       1  0.5000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
4250    0  0.923077  0.555556   2    2   9.000000       1  0.5000    0  
4251    0  0.875000  0.625000   2    2  27.000000       5  2.5000    0  
4252    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
4253    0  0.953488  0.666667   1    1  22.250000       1  0.5000    0  
4254    0  0.800000  0.666667   1    1  25.500000       1  0.5000    0  

[4255 rows x 23 columns]

In [6]:
train, test = train_test_split(ant_master, test_size=0.2)

In [7]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [8]:
x_train

wmc  dit  noc  cbo  rfc  lcom   ca  ce  npm     lcom3  loc       dam  \
691     1    1    0    2    3     0    0   2    0  2.000000   39  0.000000   
3504    5    7    0   16   18     4    1  15    1  0.916667   64  0.000000   
3512    6    6    0   14   19     3    2  12    1  0.866667   73  0.000000   
2183    6    6    0   23   22     3    6  17    1  0.866667   85  0.000000   
82      5    4    0   27   40     8    1  26    3  1.000000  233  0.000000   
...   ...  ...  ...  ...  ...   ...  ...  ..  ...       ...  ...       ...   
2960    3    4    0   35   30     3    2  33    2  2.000000  397  0.000000   
3908   25    1    0  208   86   292  183  25   22  0.958333  627  0.333333   
1491    2    2    0    4    5     1    0   4    2  2.000000   23  0.000000   
587     5    5    0   14   17     4    1  13    1  0.916667   58  0.000000   
1735    4    4    0   26   48     4    1  25    3  1.000000  241  0.000000   

      moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  
691     0  0.000000  0.000000   0    0   11.000000       0  0.0000  
3504    0  0.888889  0.666667   2    3   11.200000       1  0.6000  
3512    0  0.851852  0.733333   2    3   10.666667       1  0.6667  
2183    0  0.851852  0.733333   2    3   12.666667       1  0.6667  
82      0  0.968421  0.375000   3   10   44.600000       1  0.6000  
...   ...       ...       ...  ..  ...         ...     ...     ...  
2960    0  0.894737  0.466667   1    1  131.333333       5  2.0000  
3908    0  0.000000  0.149306   0    0   23.840000      10  2.4400  
1491    0  0.666667  0.625000   1    1   10.500000       1  0.5000  
587     0  0.880000  0.666667   2    3   10.000000       1  0.6000  
1735    0  0.966667  0.555556   3    6   58.500000       1  0.5000  

[3404 rows x 20 columns]

In [9]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
344     5    6    0   14   18     4   1  13    1  0.916667   72  0.000000   
1567    2    2    0   15   20     1   0  15    2  2.000000  164  0.000000   
3999    3    3    0   25   73     1   0  25    1  0.916667  348  0.166667   
3411    2    3    0    7   10     1   1   6    2  2.000000   38  0.000000   
4251    2    3    0   15   15     1   1  14    2  2.000000   56  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
140     2    3    0   19   15     1   2  17    2  2.000000   62  0.000000   
2395    6    3    0    5    7    15   1   4    1  2.000000   25  0.000000   
757     1    1    0    1    3     0   0   1    0  2.000000   24  0.000000   
530     2    1    0    3   10     1   0   3    2  2.000000   33  0.000000   
716    23    3    0    7   51   167   1   6   18  0.873737  326  0.444444   

      moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  
344     0  0.884615  0.666667   2    3   12.800000       1  0.6000  
1567    0  0.750000  0.600000   1    1   81.000000      13  6.5000  
3999    0  0.976190  0.750000   2    3  113.000000      11  3.6667  
3411    0  0.875000  0.625000   1    1   18.000000       3  1.5000  
4251    0  0.875000  0.625000   2    2   27.000000       5  2.5000  
...   ...       ...       ...  ..  ...         ...     ...     ...  
140     0  0.875000  0.625000   1    1   30.000000       1  0.5000  
2395    0  0.833333  0.833333   1    4    3.166667       1  0.8333  
757     0  0.000000  0.000000   0    0   11.000000       0  0.0000  
530     0  0.000000  0.666667   0    0   15.500000       2  1.0000  
716     0  0.726027  0.146245   3   22   12.782609       6  1.5217  

[851 rows x 20 columns]

In [10]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [11]:
y_train

bug
691     0
3504    0
3512    0
2183    0
82      0
...   ...
2960    0
3908    1
1491    0
587     0
1735    0

[3404 rows x 1 columns]

In [12]:
y_test

bug
344     0
1567    0
3999    0
3411    0
4251    0
...   ...
140     0
2395    0
757     0
530     0
716     0

[851 rows x 1 columns]

In [13]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [14]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [15]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [16]:
y = y_test['bug'].tolist()
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,


In [17]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv9/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
35/35 [==============================] - 1s 1ms/step - loss: 31132.1270 - mse: 31132.1270 - mae: 9.3142 - root_mean_squared_error: 176.4430 - mean_squared_logarithmic_error: 0.7390
Epoch 2/100
35/35 [==============================] - 0s 920us/step - loss: 24334.8809 - mse: 24334.8809 - mae: 7.0867 - root_mean_squared_error: 155.9964 - mean_squared_logarithmic_error: 0.4010
Epoch 3/100
35/35 [==============================] - 0s 882us/step - loss: 18994.1406 - mse: 18994.1406 - mae: 6.0302 - root_mean_squared_error: 137.8192 - mean_squared_logarithmic_error: 0.3242
Epoch 4/100
35/35 [==============================] - 0s 935us/step - loss: 14785.6055 - mse: 14785.6055 - mae: 5.2080 - root_mean_squared_error: 121.5961 - mean_squared_logarithmic_error: 0.2737
Epoch 5/100
35/35 [==============================] - 0s 859us/step - loss: 11472.4912 - mse: 11472.4912 - mae: 4.4124 - root_mean_squared_error: 107.1097 - mean_squared_logarithmic_error: 0.2390
Epoch 6/100
35/35

35/35 [==============================] - 0s 918us/step - loss: 0.1419 - mse: 0.1419 - mae: 0.0601 - root_mean_squared_error: 0.3767 - mean_squared_logarithmic_error: 0.0347
Epoch 88/100
35/35 [==============================] - 0s 971us/step - loss: 0.1419 - mse: 0.1419 - mae: 0.0601 - root_mean_squared_error: 0.3767 - mean_squared_logarithmic_error: 0.0347
Epoch 89/100
35/35 [==============================] - 0s 903us/step - loss: 0.1419 - mse: 0.1419 - mae: 0.0601 - root_mean_squared_error: 0.3767 - mean_squared_logarithmic_error: 0.0347
Epoch 90/100
35/35 [==============================] - 0s 925us/step - loss: 0.1419 - mse: 0.1419 - mae: 0.0601 - root_mean_squared_error: 0.3767 - mean_squared_logarithmic_error: 0.0347              
Epoch 91/100
35/35 [==============================] - 0s 932us/step - loss: 0.1419 - mse: 0.1419 - mae: 0.0601 - root_mean_squared_error: 0.3767 - mean_squared_logarithmic_error: 0.0347
Epoch 92/100
35/35 [==============================] - 0s 875us/step -

35/35 [==============================] - 0s 938us/step - loss: 0.1960 - mse: 0.1960 - mae: 0.0636 - root_mean_squared_error: 0.4428 - mean_squared_logarithmic_error: 0.0366
Epoch 71/100
35/35 [==============================] - 0s 856us/step - loss: 0.1878 - mse: 0.1878 - mae: 0.0633 - root_mean_squared_error: 0.4333 - mean_squared_logarithmic_error: 0.0365
Epoch 72/100
35/35 [==============================] - 0s 943us/step - loss: 0.1746 - mse: 0.1746 - mae: 0.0628 - root_mean_squared_error: 0.4178 - mean_squared_logarithmic_error: 0.0362
Epoch 73/100
35/35 [==============================] - 0s 893us/step - loss: 0.1671 - mse: 0.1671 - mae: 0.0624 - root_mean_squared_error: 0.4088 - mean_squared_logarithmic_error: 0.0361
Epoch 74/100
35/35 [==============================] - 0s 894us/step - loss: 0.1650 - mse: 0.1650 - mae: 0.0623 - root_mean_squared_error: 0.4062 - mean_squared_logarithmic_error: 0.0360
Epoch 75/100
35/35 [==============================] - 0s 924us/step - loss: 0.1563 

35/35 [==============================] - 0s 969us/step - loss: 0.1413 - mse: 0.1413 - mae: 0.0597 - root_mean_squared_error: 0.3759 - mean_squared_logarithmic_error: 0.0345
Epoch 56/100
35/35 [==============================] - 0s 992us/step - loss: 0.1413 - mse: 0.1413 - mae: 0.0596 - root_mean_squared_error: 0.3759 - mean_squared_logarithmic_error: 0.0345
Epoch 57/100
35/35 [==============================] - 0s 940us/step - loss: 0.1413 - mse: 0.1413 - mae: 0.0596 - root_mean_squared_error: 0.3759 - mean_squared_logarithmic_error: 0.0345
Epoch 58/100
35/35 [==============================] - 0s 957us/step - loss: 0.1413 - mse: 0.1413 - mae: 0.0596 - root_mean_squared_error: 0.3759 - mean_squared_logarithmic_error: 0.0345
Epoch 59/100
35/35 [==============================] - 0s 982us/step - loss: 0.1413 - mse: 0.1413 - mae: 0.0596 - root_mean_squared_error: 0.3759 - mean_squared_logarithmic_error: 0.0345
Epoch 60/100
35/35 [==============================] - 0s 926us/step - loss: 0.1413 

35/35 [==============================] - 0s 923us/step - loss: 0.1413 - mse: 0.1413 - mae: 0.0596 - root_mean_squared_error: 0.3759 - mean_squared_logarithmic_error: 0.0345
Epoch 100/100
27/27 [==============================] - 0s 633us/step - loss: 0.1868 - mse: 0.1868 - mae: 0.0717 - root_mean_squared_error: 0.4322 - mean_squared_logarithmic_error: 0.0422
score of 2th run is: 0.1868390142917633, 0.1868390142917633, 0.07168037444353104, 0.4322488009929657, 0.04217567667365074
Run no: 3
Epoch 1/100
35/35 [==============================] - 0s 2ms/step - loss: 0.1415 - mse: 0.1415 - mae: 0.0605 - root_mean_squared_error: 0.3761 - mean_squared_logarithmic_error: 0.0346
Epoch 2/100
35/35 [==============================] - 0s 902us/step - loss: 0.1414 - mse: 0.1414 - mae: 0.0603 - root_mean_squared_error: 0.3761 - mean_squared_logarithmic_error: 0.0346
Epoch 3/100
35/35 [==============================] - 0s 900us/step - loss: 0.1414 - mse: 0.1414 - mae: 0.0602 - root_mean_squared_error: 0.3

...

35/35 [==============================] - 0s 1ms/step - loss: 0.1277 - mse: 0.1277 - mae: 0.0938 - root_mean_squared_error: 0.3574 - mean_squared_logarithmic_error: 0.0317
Epoch 20/100
35/35 [==============================] - 0s 989us/step - loss: 0.1270 - mse: 0.1270 - mae: 0.0885 - root_mean_squared_error: 0.3563 - mean_squared_logarithmic_error: 0.0312
Epoch 21/100
35/35 [==============================] - 0s 938us/step - loss: 0.1269 - mse: 0.1269 - mae: 0.0912 - root_mean_squared_error: 0.3563 - mean_squared_logarithmic_error: 0.0313
Epoch 22/100
35/35 [==============================] - 0s 995us/step - loss: 0.1268 - mse: 0.1268 - mae: 0.0894 - root_mean_squared_error: 0.3561 - mean_squared_logarithmic_error: 0.0310
Epoch 23/100
35/35 [==============================] - 0s 999us/step - loss: 0.1272 - mse: 0.1272 - mae: 0.0913 - root_mean_squared_error: 0.3566 - mean_squared_logarithmic_error: 0.0314
Epoch 24/100
35/35 [==============================] - 0s 1ms/step - loss: 0.1265 - ms

35/35 [==============================] - 0s 905us/step - loss: 411.4760 - mse: 411.4760 - mae: 6.5541 - root_mean_squared_error: 20.2849 - mean_squared_logarithmic_error: 2.9942
Epoch 5/100
35/35 [==============================] - 0s 868us/step - loss: 188.8703 - mse: 188.8703 - mae: 4.0034 - root_mean_squared_error: 13.7430 - mean_squared_logarithmic_error: 1.9188
Epoch 6/100
35/35 [==============================] - 0s 928us/step - loss: 122.0233 - mse: 122.0233 - mae: 2.7086 - root_mean_squared_error: 11.0464 - mean_squared_logarithmic_error: 1.3040
Epoch 7/100
35/35 [==============================] - 0s 874us/step - loss: 102.0392 - mse: 102.0392 - mae: 2.0408 - root_mean_squared_error: 10.1014 - mean_squared_logarithmic_error: 0.9578
Epoch 8/100
35/35 [==============================] - 0s 882us/step - loss: 89.2346 - mse: 89.2346 - mae: 1.5831 - root_mean_squared_error: 9.4464 - mean_squared_logarithmic_error: 0.7171
Epoch 9/100
35/35 [==============================] - 0s 902us/ste

35/35 [==============================] - 0s 1ms/step - loss: 0.2015 - mse: 0.2015 - mae: 0.0640 - root_mean_squared_error: 0.4489 - mean_squared_logarithmic_error: 0.0352
Epoch 92/100
35/35 [==============================] - 0s 972us/step - loss: 0.1929 - mse: 0.1929 - mae: 0.0636 - root_mean_squared_error: 0.4393 - mean_squared_logarithmic_error: 0.0351
Epoch 93/100
35/35 [==============================] - 0s 979us/step - loss: 0.1857 - mse: 0.1857 - mae: 0.0633 - root_mean_squared_error: 0.4309 - mean_squared_logarithmic_error: 0.0350
Epoch 94/100
35/35 [==============================] - 0s 1ms/step - loss: 0.1792 - mse: 0.1792 - mae: 0.0629 - root_mean_squared_error: 0.4233 - mean_squared_logarithmic_error: 0.0349
Epoch 95/100
35/35 [==============================] - 0s 1ms/step - loss: 0.1752 - mse: 0.1752 - mae: 0.0627 - root_mean_squared_error: 0.4186 - mean_squared_logarithmic_error: 0.0349
Epoch 96/100
35/35 [==============================] - 0s 1ms/step - loss: 0.1685 - mse: 0

35/35 [==============================] - 0s 992us/step - loss: 0.1410 - mse: 0.1410 - mae: 0.0598 - root_mean_squared_error: 0.3755 - mean_squared_logarithmic_error: 0.0343
Epoch 77/100
35/35 [==============================] - 0s 989us/step - loss: 0.1410 - mse: 0.1410 - mae: 0.0598 - root_mean_squared_error: 0.3754 - mean_squared_logarithmic_error: 0.0343
Epoch 78/100
35/35 [==============================] - 0s 988us/step - loss: 0.1409 - mse: 0.1409 - mae: 0.0598 - root_mean_squared_error: 0.3754 - mean_squared_logarithmic_error: 0.0343
Epoch 79/100
35/35 [==============================] - 0s 1ms/step - loss: 0.1409 - mse: 0.1409 - mae: 0.0597 - root_mean_squared_error: 0.3754 - mean_squared_logarithmic_error: 0.0343
Epoch 80/100
35/35 [==============================] - 0s 995us/step - loss: 0.1409 - mse: 0.1409 - mae: 0.0597 - root_mean_squared_error: 0.3754 - mean_squared_logarithmic_error: 0.0343
Epoch 81/100
35/35 [==============================] - 0s 975us/step - loss: 0.1409 - 

35/35 [==============================] - 0s 1ms/step - loss: 57.7288 - mse: 57.7288 - mae: 1.1479 - root_mean_squared_error: 7.5979 - mean_squared_logarithmic_error: 0.4659
Epoch 60/100
35/35 [==============================] - 0s 1ms/step - loss: 54.5618 - mse: 54.5618 - mae: 1.0856 - root_mean_squared_error: 7.3866 - mean_squared_logarithmic_error: 0.4400
Epoch 61/100
35/35 [==============================] - 0s 1ms/step - loss: 51.0394 - mse: 51.0394 - mae: 1.0383 - root_mean_squared_error: 7.1442 - mean_squared_logarithmic_error: 0.4213
Epoch 62/100
35/35 [==============================] - 0s 1ms/step - loss: 48.4717 - mse: 48.4717 - mae: 0.9842 - root_mean_squared_error: 6.9622 - mean_squared_logarithmic_error: 0.3991
Epoch 63/100
35/35 [==============================] - 0s 995us/step - loss: 45.4319 - mse: 45.4319 - mae: 0.9389 - root_mean_squared_error: 6.7403 - mean_squared_logarithmic_error: 0.3810
Epoch 64/100
35/35 [==============================] - 0s 931us/step - loss: 42.88

In [18]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv9/'

In [19]:
total_training_time

[4.308176279067993,
 4.240428447723389,
 4.1732447147369385,
 4.148448705673218,
 4.285920143127441,
 4.194425582885742,
 4.034336805343628,
 4.039971828460693,
 4.070208787918091,
 4.00667667388916,
 4.074457406997681,
 4.027095556259155,
 3.992729425430298,
 4.075406789779663,
 4.02714467048645,
 4.112433910369873,
 4.199810266494751,
 4.197949647903442,
 4.1282877922058105,
 4.035897493362427,
 4.046997785568237,
 4.063279390335083,
 4.133407354354858,
 4.046909809112549,
 4.1382880210876465,
 4.153802394866943,
 4.28960108757019,
 4.12898063659668,
 4.518444061279297,
 4.389992713928223]

In [20]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 4.1427584727605185


In [21]:
total_training_loss_history

[{'loss': [31132.126953125,
   24334.880859375,
   18994.140625,
   14785.60546875,
   11472.4912109375,
   8645.478515625,
   6336.32373046875,
   4608.873046875,
   3393.31884765625,
   2353.376953125,
   1679.9525146484375,
   1147.099365234375,
   762.7626342773438,
   489.021240234375,
   310.20904541015625,
   207.94454956054688,
   125.5596923828125,
   80.07271575927734,
   51.98841094970703,
   32.99034118652344,
   21.98974609375,
   16.0478458404541,
   10.528106689453125,
   7.557140827178955,
   5.059238433837891,
   3.568135976791382,
   2.3974666595458984,
   1.5905256271362305,
   1.091598629951477,
   0.7660142779350281,
   0.5183870196342468,
   0.4473506510257721,
   0.2786679267883301,
   0.22315266728401184,
   0.20088902115821838,
   0.16906102001667023,
   0.15774931013584137,
   0.1514228731393814,
   0.1474492996931076,
   0.1447499394416809,
   0.14354218542575836,
   0.14328041672706604,
   0.1424325555562973,
   0.14230240881443024,
   0.1421726793050766,
  

In [22]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 1.568690507610639
avg mae of training: 0.07479519546031951


In [23]:
total_scores

[[0.6337605118751526,
  0.6337605118751526,
  0.09345202893018723,
  0.7960907816886902,
  0.04824599251151085],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [11.03194808959961,
  11.03194808959961,
  0.22233463823795319,
  3.3214375972747803,
  0.07776

In [1]:
a = [[0.6337605118751526,
  0.6337605118751526,
  0.09345202893018723,
  0.7960907816886902,
  0.04824599251151085],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [11.03194808959961,
  11.03194808959961,
  0.22233463823795319,
  3.3214375972747803,
  0.07776869833469391],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1902959793806076,
  0.1902959793806076,
  0.07369587570428848,
  0.43622928857803345,
  0.04334809631109238],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [1.299893856048584,
  1.299893856048584,
  0.1119089424610138,
  1.1401288509368896,
  0.0589740127325058],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [2.625580310821533,
  2.625580310821533,
  0.14703282713890076,
  1.6203643083572388,
  0.07237669825553894],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.18684108555316925,
  0.18684108555316925,
  0.07173177599906921,
  0.4322511851787567,
  0.042177680879831314],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.41808170080184937,
  0.41808170080184937,
  0.09677328169345856,
  0.646592378616333,
  0.056527260690927505],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.16680774092674255,
  0.16680774092674255,
  0.09866085648536682,
  0.40842103958129883,
  0.03762594237923622],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [0.1868390142917633,
  0.1868390142917633,
  0.07168037444353104,
  0.4322488009929657,
  0.04217567667365074],
 [1.0310933589935303,
  1.0310933589935303,
  0.1332574486732483,
  1.0154277086257935,
  0.06302836537361145]]

In [2]:
test_mae = []
for i in range(30):
    test_mae.append(a[i][2])

In [3]:
test_mae

[0.09345202893018723,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.22233463823795319,
 0.07168037444353104,
 0.07369587570428848,
 0.07168037444353104,
 0.07168037444353104,
 0.1119089424610138,
 0.07168037444353104,
 0.14703282713890076,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.07173177599906921,
 0.07168037444353104,
 0.09677328169345856,
 0.07168037444353104,
 0.07168037444353104,
 0.07168037444353104,
 0.09866085648536682,
 0.07168037444353104,
 0.07168037444353104,
 0.1332574486732483]

In [27]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.7169307311375935
avg mae: 0.08513785128792127
avg are: 0.04213082566857338


In [28]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.14130797237157822
median testing mse: 0.1868390142917633


In [29]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.05970717780292034
median testing mae: 0.07168037444353104


In [30]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.03760281950235367


In [31]:
test_ARE

[0.058754407,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.14218566,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.037015278,
 0.03760282,
 0.03760282,
 0.04829612]

In [6]:
sum([0.058754407,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.14218566,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.03760282,
 0.037015278,
 0.03760282,
 0.03760282,
 0.04829612][:10])/10

0.050176262699999995